In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import csv
import pandas as pd
from googlesearch import search
import wikipedia

In [ ]:
companies = pd.read_csv("training_data.csv", header = None)
companies = companies[0].tolist()

In [ ]:
def get_soup(url):
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    soup = BeautifulSoup(response, "html.parser")
    return(soup)

In [ ]:
def get_first_url(query):
    for url in search(query, tld="com.sg", num=1, stop=1, pause=3):
        return url

In [ ]:
def get_about_us(companies):
    company_details = {}
    for company in companies:
        company_details[company] = {}
        query = company +" bloomberg snapshot"
        url=get_first_url(query)
        if url[:58] == "https://www.bloomberg.com/research/stocks/private/snapshot":
            soup = get_soup(url)
            about=[]
            add_line = soup.find("p",{"id": "bDesc"})
            #about=add_line.find("p")
            about_us=add_line.text.strip()
            #print(company, "\n",about_us) 
            company_details[company]["About_Us"]=about_us
        else:
            query = company + " wikipedia singapore"
            url=get_first_url(query)
            if url[:24]=="https://en.wikipedia.org":
                try:
                    about_us=wikipedia.summary(company)
                    #print(company,"\n",about_us)
                    company_details[company]["About_Us"]=about_us
                except:
                    company_details[company]["About_Us"]="NA"
                    continue
            else:
                query = company +" pitchbook"
                url=get_first_url(query)
                if url[:38] == "https://pitchbook.com/profiles/company":
                    soup = get_soup(url)
                    about = []
                    try:
                        add_line = soup.find("div", attrs={"class": "XL-12 column general-info__description shift-bottom-XL-30 shift-top-XL-35"})
                        abt=add_line.find("p")
                        about_us = abt.text.strip()
                        #print(company,"\n", about_us)
                        company_details[company]["About_Us"]=about_us
                    except:
                        company_details[company]["About_Us"]="NA"
                        continue
                    
                else:
                    company_details[company]["About_Us"]= "NA"
    return company_details

In [ ]:
get_about_us(companies)